<a href="https://colab.research.google.com/github/Dhanush834/AusgutAI/blob/main/UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai portkey-ai transformers torch sentence-transformers scikit-learn numpy gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 380.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.9/460.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import gradio as gr
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL

class IndianLawRAGAgent:
    def __init__(self, portkey_api_key, portkey_virtual_key, bert_model_name, knowledge_base_path):
        self.client = OpenAI(
            api_key="dummy",
            base_url=PORTKEY_GATEWAY_URL,
            default_headers=createHeaders(
                provider="openai",
                api_key=portkey_api_key,
                virtual_key=portkey_virtual_key
            )
        )

        # Load BERT model and tokenizer from Hugging Face
        self.bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
        self.bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

        # Load knowledge base
        with open(knowledge_base_path, 'r') as f:
            data = json.load(f)
            self.knowledge_base = data["knowledge_base"]

        # Encode documents
        self.document_embeddings = self.sentence_model.encode([doc['content'] for doc in self.knowledge_base])

    def retrieve_relevant_documents(self, query, top_k=3):
        query_embedding = self.sentence_model.encode([query])
        similarities = cosine_similarity(query_embedding, self.document_embeddings)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [self.knowledge_base[i] for i in top_indices]

    def answer_legal_question(self, question, context):
        inputs = self.bert_tokenizer(question, context, return_tensors="pt")
        outputs = self.bert_model(**inputs)

        answer_start = outputs.start_logits.argmax()
        answer_end = outputs.end_logits.argmax() + 1
        answer = self.bert_tokenizer.convert_tokens_to_string(
            self.bert_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
        )
        return answer

    def generate_legal_advice(self, startup_type, situation):
        relevant_docs = self.retrieve_relevant_documents(f"{startup_type} {situation}")
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""As a legal expert, provide advice for an Indian {startup_type} startup in the following situation: {situation}

        Relevant legal information:
        {context}

        Advice:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal assistant specializing in Indian startup law."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def summarize_law(self, law_name):
        relevant_docs = self.retrieve_relevant_documents(law_name)
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""Summarize the key points of the Indian law: {law_name}, especially as it pertains to startups.

        Relevant legal information:
        {context}

        Summary:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal assistant specializing in Indian law summaries."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def check_compliance(self, startup_description):
        relevant_docs = self.retrieve_relevant_documents(startup_description)
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""Given the following Indian startup description, list the key compliance requirements and potential legal issues to be aware of: {startup_description}

        Relevant legal information:
        {context}

        Compliance requirements and potential legal issues:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal assistant specializing in compliance for Indian startups."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

def create_ui():
    agent = IndianLawRAGAgent(
        portkey_api_key="1+YM2sBEgaZWe45bMOq2huic1uyF",
        portkey_virtual_key="015-openai-40bada",
        bert_model_name="law-ai/InLegalBERT",
        knowledge_base_path="test.json"
    )

    def legal_question_interface(question, context):
        return agent.answer_legal_question(question, context)

    def legal_advice_interface(startup_type, situation):
        return agent.generate_legal_advice(startup_type, situation)

    def law_summary_interface(law_name):
        return agent.summarize_law(law_name)

    def compliance_check_interface(startup_description):
        return agent.check_compliance(startup_description)

    with gr.Blocks() as demo:
        gr.Markdown("# Indian Law RAG Agent")

        with gr.Tab("Answer Legal Question"):
            question = gr.Textbox(label="Enter your legal question")
            context = gr.Textbox(label="Enter the legal context")
            answer_button = gr.Button("Get Answer")
            answer_output = gr.Textbox(label="Answer", interactive=False)
            answer_button.click(legal_question_interface, inputs=[question, context], outputs=answer_output)

        with gr.Tab("Generate Legal Advice"):
            startup_type = gr.Textbox(label="Enter the startup type")
            situation = gr.Textbox(label="Describe the situation")
            advice_button = gr.Button("Get Advice")
            advice_output = gr.Textbox(label="Legal Advice", interactive=False)
            advice_button.click(legal_advice_interface, inputs=[startup_type, situation], outputs=advice_output)

        with gr.Tab("Summarize Law"):
            law_name = gr.Textbox(label="Enter the law name")
            summary_button = gr.Button("Get Summary")
            summary_output = gr.Textbox(label="Law Summary", interactive=False)
            summary_button.click(law_summary_interface, inputs=[law_name], outputs=summary_output)

        with gr.Tab("Check Compliance"):
            startup_description = gr.Textbox(label="Enter the startup description")
            compliance_button = gr.Button("Check Compliance")
            compliance_output = gr.Textbox(label="Compliance Check", interactive=False)
            compliance_button.click(compliance_check_interface, inputs=[startup_description], outputs=compliance_output)

    demo.launch()

if __name__ == "__main__":
    create_ui()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://32b5291f3c348ddef0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:
import json
import numpy as np
import gradio as gr
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL

class IndianLawRAGAgent:
    def __init__(self, portkey_api_key, portkey_virtual_key, bert_model_name, knowledge_base_path):
        self.client = OpenAI(
            api_key="dummy",
            base_url=PORTKEY_GATEWAY_URL,
            default_headers=createHeaders(
                provider="openai",
                api_key=portkey_api_key,
                virtual_key=portkey_virtual_key
            )
        )

        # Load BERT model and tokenizer from Hugging Face
        self.bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
        self.bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

        # Load knowledge base
        with open(knowledge_base_path, 'r') as f:
            data = json.load(f)
            self.knowledge_base = data["knowledge_base"]

        # Encode documents
        self.document_embeddings = self.sentence_model.encode([doc['content'] for doc in self.knowledge_base])

    def retrieve_relevant_documents(self, query, top_k=3):
        query_embedding = self.sentence_model.encode([query])
        similarities = cosine_similarity(query_embedding, self.document_embeddings)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [self.knowledge_base[i] for i in top_indices]

    def answer_legal_question(self, question, context):
        inputs = self.bert_tokenizer(question, context, return_tensors="pt")
        outputs = self.bert_model(**inputs)

        answer_start = outputs.start_logits.argmax()
        answer_end = outputs.end_logits.argmax() + 1
        answer = self.bert_tokenizer.convert_tokens_to_string(
            self.bert_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
        )
        return answer

    def generate_legal_advice(self, startup_type, situation):
        relevant_docs = self.retrieve_relevant_documents(f"{startup_type} {situation}")
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""As a legal expert, provide advice for an Indian {startup_type} startup in the following situation: {situation}

        Relevant legal information:
        {context}

        Advice:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal assistant specializing in Indian startup law."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def summarize_law(self, law_name):
        relevant_docs = self.retrieve_relevant_documents(law_name)
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""Summarize the key points of the Indian law: {law_name}, especially as it pertains to startups.

        Relevant legal information:
        {context}

        Summary:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal assistant specializing in Indian law summaries."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def check_compliance(self, startup_description):
        relevant_docs = self.retrieve_relevant_documents(startup_description)
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""Given the following Indian startup description, list the key compliance requirements and potential legal issues to be aware of: {startup_description}

        Relevant legal information:
        {context}

        Compliance requirements and potential legal issues:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal assistant specializing in compliance for Indian startups."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def handle_user_input(self, user_input):
        # Step 1: Summarize the user input
        summarization_prompt = f"""Summarize the following legal input to extract key points:

        Input: {user_input}

        Summary:"""

        summarization_response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI summarization assistant."},
                {"role": "user", "content": summarization_prompt}
            ]
        )
        summarized_input = summarization_response.choices[0].message.content.strip()

        # Step 2: Classify the user input into a type (e.g., 'legal question', 'generate advice', etc.)
        classification_prompt = f"""Classify the following summarized legal input into one of these categories:
        'legal question', 'generate advice', 'summarize law', 'check compliance'.

        Summary: {summarized_input}

        Classification:"""

        classification_response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI classification assistant."},
                {"role": "user", "content": classification_prompt}
            ]
        )
        classification = classification_response.choices[0].message.content.strip().lower()

        # Step 3: Based on classification, automatically generate the legal context or invoke appropriate function
        if "legal question" in classification:
            legal_context_prompt = f"""Generate the legal context for the following legal question:

            Question: {summarized_input}

            Legal Context:"""

            legal_context_response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are an AI legal context assistant."},
                    {"role": "user", "content": legal_context_prompt}
                ]
            )
            legal_context = legal_context_response.choices[0].message.content.strip()
            return f"Function called: answer_legal_question\nResponse: {self.answer_legal_question(summarized_input, legal_context)}"

        elif "generate advice" in classification:
            startup_type_prompt = f"""Identify the startup type based on the following input:

            Input: {summarized_input}

            Startup Type:"""

            startup_type_response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are an AI startup classification assistant."},
                    {"role": "user", "content": startup_type_prompt}
                ]
            )
            startup_type = startup_type_response.choices[0].message.content.strip()
            return f"Function called: generate_legal_advice\nResponse: {self.generate_legal_advice(startup_type, summarized_input)}"

        elif "summarize law" in classification:
            return f"Function called: summarize_law\nResponse: {self.summarize_law(summarized_input)}"

        elif "check compliance" in classification:
            return f"Function called: check_compliance\nResponse: {self.check_compliance(summarized_input)}"

        else:
            return "Sorry, I couldn't classify your input. Please try again."

def gradio_interface():
    agent = IndianLawRAGAgent(
        portkey_api_key="1+YM2sBEgaZWe45bMOq2huic1uyF",
        portkey_virtual_key="015-openai-40bada",
        bert_model_name="law-ai/InLegalBERT",
        knowledge_base_path="test.json"
    )

    def process_input(user_input):
        return agent.handle_user_input(user_input)

    iface = gr.Interface(
        fn=process_input,
        inputs="text",
        outputs="text",
        title="Indian Law RAG Agent",
        description="Ask legal questions or get advice related to Indian startup laws.",
        examples=[
            ["How can I register a startup in India?"],
            ["What are the compliance requirements for an IT startup?"],
            ["Summarize the Indian contract law."],
            ["Check compliance for a new e-commerce startup."]
        ]
    )

    iface.launch()

if __name__ == "__main__":
    gradio_interface()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b59fe4892221242287.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [26]:
pip install docx python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.1 MB/s eta 0:00:00


In [28]:
import json
import numpy as np
import gradio as gr
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document
import io
from openai import OpenAI

class NaturalLanguageComplianceChecker:
    def __init__(self, portkey_api_key, portkey_virtual_key, bert_model_name, knowledge_base_path):
        self.client = OpenAI(
            api_key="dummy",
            base_url=PORTKEY_GATEWAY_URL,  # Replace with actual URL including http:// or https://
            default_headers=createHeaders(
                provider="openai",
                api_key=portkey_api_key,
                virtual_key=portkey_virtual_key
            )
        )

        # Load BERT model and tokenizer from Hugging Face
        self.bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
        self.bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

        # Load knowledge base
        with open(knowledge_base_path, 'r', errors="ignore") as f:
            data = json.load(f)
            self.knowledge_base = data["knowledge_base"]

        # Encode documents for legal validation
        self.document_embeddings = self.sentence_model.encode([doc['content'] for doc in self.knowledge_base])

    def retrieve_relevant_documents(self, query, top_k=3):
        query_embedding = self.sentence_model.encode([query])
        similarities = cosine_similarity(query_embedding, self.document_embeddings)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [self.knowledge_base[i] for i in top_indices]

    def validate_contract(self, contract_text):
        relevant_docs = self.retrieve_relevant_documents("contract clause compliance")
        context = "\n".join([doc['content'] for doc in relevant_docs])

        prompt = f"""Validate the following contract content for legal compliance:

        Contract Text: {contract_text}

        Relevant legal information:
        {context}

        Compliance validation and suggestions:"""

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an AI legal compliance assistant specializing in Indian law."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

def upload_contract_file(file):
    # Read the uploaded .docx file
    doc = Document(io.BytesIO(file))
    contract_text = '\n'.join([para.text for para in doc.paragraphs])
    return contract_text

def validate_uploaded_file(file):
    agent = NaturalLanguageComplianceChecker(
        portkey_api_key="1+YM2sBEgaZWe45bMOq2huic1uyF",
        portkey_virtual_key="015-openai-40bada",
        bert_model_name="law-ai/InLegalBERT",
        knowledge_base_path="test.json"
    )

    # Get the contract text from the uploaded file
    contract_text = upload_contract_file(file)

    # Validate the contract and return the compliance result
    validation_result = agent.validate_contract(contract_text)
    return validation_result

# Define Gradio UI
def gradio_interface():
    interface = gr.Interface(
        fn=validate_uploaded_file,
        inputs=gr.File(type="binary", label="Upload .docx Contract File"),
        outputs=gr.Textbox(label="Compliance Validation and Suggestions"),
        title="Contract Compliance Checker",
        description="Upload a contract file in .docx format to get legal compliance validation and suggestions."
    )
    interface.launch()

if __name__ == "__main__":
    gradio_interface()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://33a3bb4cf7e7bb67f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [30]:
import json
import random
import numpy as np
import gradio as gr
from datetime import datetime, timedelta
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
from docx import Document
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL

class ContractGenerator:
    def __init__(self, portkey_api_key, portkey_virtual_key, bert_model_name, knowledge_base_path):
        self.client = OpenAI(
            api_key="dummy",
            base_url=PORTKEY_GATEWAY_URL,
            default_headers=createHeaders(
                provider="openai",
                api_key=portkey_api_key,
                virtual_key=portkey_virtual_key
            )
        )
        # Load BERT model and tokenizer from Hugging Face
        self.bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
        self.bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

        # Load knowledge base
        with open(knowledge_base_path, 'r') as f:
            data = json.load(f)
            self.company_name = data["company_name"]
            self.knowledge_base = data["knowledge_base"]

        # Encode documents
        self.document_embeddings = self.sentence_model.encode([doc['content'] for doc in self.knowledge_base])

    def generate_contract_prompt(self, contract_type, details):
        if contract_type == '1':  # Employment Contract
            job_title, salary_info, responsibilities = details
            prompt = f"""
            Create a legal employment contract based on the following information:

            Company: {self.company_name}
            Job Title: {job_title}
            Employee: {self.employee_name}
            Start Date: {self.start_date}
            Salary Information: {salary_info}
            Key Responsibilities: {responsibilities}

            The contract should include:
            - Names of the parties
            - Effective date
            - Job title and department
            - Salary and benefits
            - Key responsibilities
            - Standard work terms
            - Signatures

            Contract:
            """
        elif contract_type == '2':  # NDA
            parties, confidential_info = details
            prompt = f"""
            Create a Non-Disclosure Agreement (NDA) based on the following information:

            Company: {self.company_name}
            Parties: {parties}
            Confidential Information: {confidential_info}

            The NDA should include:
            - Definition of confidential information
            - Obligations of the parties
            - Term and duration of the NDA
            - Non-use and non-disclosure clauses
            - Termination conditions
            - Signatures

            NDA:
            """
        elif contract_type == '3':  # Partnership Agreement
            partners, partnership_terms = details
            prompt = f"""
            Create a Partnership Agreement based on the following information:

            Company: {self.company_name}
            Partners: {partners}
            Key Terms: {partnership_terms}

            The agreement should include:
            - Names of the parties
            - Partnership terms
            - Responsibilities of the partners
            - Profit and loss sharing terms
            - Dispute resolution
            - Termination conditions
            - Signatures

            Partnership Agreement:
            """
        return prompt

    def generate_contract(self, contract_type, details, employee_name, start_date):
        # Store general questions
        self.employee_name = employee_name
        self.start_date = start_date

        prompt = self.generate_contract_prompt(contract_type, details)

        # Use OpenAI to generate the contract text
        response = self.client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a legal expert specializing in drafting contracts."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def save_contract_to_file(self, contract_text, filename="contract.docx"):
        doc = Document()
        doc.add_paragraph(contract_text)
        doc.add_paragraph("[It is advisable that both parties have this document reviewed by their respective legal advisors before signing.]")
        doc.save(filename)

    def generate_and_save_contract(self, contract_type, details, employee_name, start_date):
        contract_text = self.generate_contract(contract_type, details, employee_name, start_date)
        if contract_text:
            filename = f"generated_{self.company_name.replace(' ', '').lower()}_contract_{contract_type}.docx"
            self.save_contract_to_file(contract_text, filename)
            return contract_text, filename

# Define Gradio UI
def gradio_interface():
    agent = ContractGenerator(
        portkey_api_key="1+YM2sBEgaZWe45bMOq2huic1uyF",
        portkey_virtual_key="015-openai-40bada",
        bert_model_name="law-ai/InLegalBERT",
        knowledge_base_path="test.json"
    )

    def handle_contract_generation(contract_type, employee_name, start_date, job_title, salary_info, responsibilities, parties, confidential_info, partners, partnership_terms):
        if contract_type == '1':
            details = (job_title, salary_info, responsibilities)
        elif contract_type == '2':
            details = (parties, confidential_info)
        elif contract_type == '3':
            details = (partners, partnership_terms)
        else:
            return "Invalid contract type."

        contract_text, filename = agent.generate_and_save_contract(contract_type, details, employee_name, start_date)
        return contract_text, filename

    contract_type_input = gr.Radio(choices=['1', '2', '3'], label="Select Contract Type")  # Corrected here

    employee_name_input = gr.Textbox(label="Employee Name (for Employment Contract)")
    start_date_input = gr.Textbox(label="Contract Start Date (YYYY-MM-DD)")
    job_title_input = gr.Textbox(label="Job Title (for Employment Contract)")
    salary_info_input = gr.Textbox(label="Salary Information (for Employment Contract)")
    responsibilities_input = gr.Textbox(label="Responsibilities (for Employment Contract)")
    parties_input = gr.Textbox(label="Parties (for NDA)")
    confidential_info_input = gr.Textbox(label="Confidential Information (for NDA)")
    partners_input = gr.Textbox(label="Partners (for Partnership Agreement)")
    partnership_terms_input = gr.Textbox(label="Partnership Terms (for Partnership Agreement)")

    # Create Gradio interface
    interface = gr.Interface(
        fn=handle_contract_generation,
        inputs=[
            contract_type_input,
            employee_name_input,
            start_date_input,
            job_title_input,
            salary_info_input,
            responsibilities_input,
            parties_input,
            confidential_info_input,
            partners_input,
            partnership_terms_input
        ],
        outputs=["text", "file"],
        title="Contract Generator",
        description="Generate Employment Contracts, NDAs, and Partnership Agreements based on input data."
    )

    interface.launch()

if __name__ == "__main__":
    gradio_interface()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce6d4170ab02ecf6ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
